# Credit Strategy Simulator

This notebook implements "What-if" scenarios for credit strategy optimization:

**Strategy Levers:**
- Acceptance score thresholds
- Credit limit policies
- Income band targeting
- Regional focus

**Optimization Goals:**
- Maximize profitability (ROA, RoRWA)
- Minimize portfolio risk
- Balance growth vs. quality trade-offs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette('viridis')

In [ ]:
# Load portfolio data and risk metrics
portfolio_df = pd.read_csv('../data/credit_portfolio.csv')
risk_df = pd.read_csv('../data/portfolio_with_risk_metrics.csv')

print(f"Portfolio dataset: {len(portfolio_df):,} customers")
print(f"Risk dataset: {len(risk_df):,} customers")

# Merge datasets for comprehensive strategy analysis
strategy_df = portfolio_df.merge(risk_df[['customer_id', 'pd_12m', 'lgd', 'ead', 'expected_loss', 'risk_score', 'risk_segment']], 
                                on='customer_id', how='left')

print(f"Combined dataset: {len(strategy_df):,} customers")

# Current portfolio baseline metrics
approved_df = strategy_df[strategy_df['acceptance_decision'] == 'Approved'].copy()
baseline_metrics = {
    'customers': len(approved_df),
    'approval_rate': len(approved_df) / len(strategy_df) * 100,
    'portfolio_balance': approved_df['balance'].sum(),
    'avg_credit_limit': approved_df['credit_limit'].mean(),
    'delinquency_rate': (approved_df['delinquency_status'] > 0).mean() * 100,
    'expected_loss': approved_df['expected_loss'].sum(),
    'loss_rate': approved_df['expected_loss'].sum() / approved_df['balance'].sum() * 100
}

print("\n=== BASELINE PORTFOLIO METRICS ===")
for metric, value in baseline_metrics.items():
    if isinstance(value, (int, float)):
        if 'rate' in metric or 'loss_rate' in metric:
            print(f"{metric}: {value:.2f}%")
        elif 'balance' in metric or 'limit' in metric or 'loss' in metric:
            print(f"{metric}: £{value/1e6 if value > 1e6 else value/1e3:.1f}{'M' if value > 1e6 else 'k'}")
        else:
            print(f"{metric}: {value:,.0f}")

## 1. Strategy Simulation Framework

Core functions to simulate different credit strategies and calculate business impact.

In [ ]:
class CreditStrategySimulator:
    """
    Simulate credit strategies and calculate business impact
    """
    
    def __init__(self, data):
        self.data = data.copy()
        self.baseline = self._calculate_baseline()
    
    def _calculate_baseline(self):
        """Calculate baseline portfolio metrics"""
        approved = self.data[self.data['acceptance_decision'] == 'Approved']
        return {
            'customers': len(approved),
            'approval_rate': len(approved) / len(self.data) * 100,
            'portfolio_balance': approved['balance'].sum(),
            'credit_limits': approved['credit_limit'].sum(),
            'delinquency_rate': (approved['delinquency_status'] > 0).mean() * 100,
            'expected_loss': approved['expected_loss'].sum(),
            'loss_rate': approved['expected_loss'].sum() / approved['balance'].sum() * 100 if approved['balance'].sum() > 0 else 0,
            'avg_score': approved['application_score'].mean(),
            'revenue_estimate': approved['balance'].sum() * 0.18 / 12,  # Assume 18% APR monthly
            'net_income': approved['balance'].sum() * 0.18 / 12 - approved['expected_loss'].sum(),
            'roa': (approved['balance'].sum() * 0.18 / 12 - approved['expected_loss'].sum()) / approved['balance'].sum() * 100 if approved['balance'].sum() > 0 else 0
        }
    
    def simulate_score_threshold_strategy(self, min_score):
        """
        Simulate changing the minimum acceptance score threshold
        """
        simulated_data = self.data.copy()
        
        # Apply new acceptance criteria
        simulated_data['new_acceptance'] = 'Declined'
        eligible_mask = simulated_data['application_score'] >= min_score
        simulated_data.loc[eligible_mask, 'new_acceptance'] = 'Approved'
        
        # Calculate metrics for new strategy
        approved = simulated_data[simulated_data['new_acceptance'] == 'Approved']
        
        if len(approved) == 0:
            return self._empty_result()
        
        return {
            'strategy': f'Score Threshold >= {min_score}',
            'customers': len(approved),
            'approval_rate': len(approved) / len(self.data) * 100,
            'portfolio_balance': approved['balance'].sum(),
            'credit_limits': approved['credit_limit'].sum(),
            'delinquency_rate': (approved['delinquency_status'] > 0).mean() * 100,
            'expected_loss': approved['expected_loss'].sum(),
            'loss_rate': approved['expected_loss'].sum() / approved['balance'].sum() * 100 if approved['balance'].sum() > 0 else 0,
            'avg_score': approved['application_score'].mean(),
            'revenue_estimate': approved['balance'].sum() * 0.18 / 12,
            'net_income': approved['balance'].sum() * 0.18 / 12 - approved['expected_loss'].sum(),
            'roa': (approved['balance'].sum() * 0.18 / 12 - approved['expected_loss'].sum()) / approved['balance'].sum() * 100 if approved['balance'].sum() > 0 else 0
        }
    
    def simulate_limit_multiplier_strategy(self, multiplier):
        """
        Simulate changing credit limit policy (multiply all limits by factor)
        """
        simulated_data = self.data.copy()
        approved_mask = simulated_data['acceptance_decision'] == 'Approved'
        
        # Adjust credit limits
        simulated_data.loc[approved_mask, 'credit_limit'] = simulated_data.loc[approved_mask, 'credit_limit'] * multiplier
        
        # Assume utilization stays similar, so balance increases proportionally (capped at new limit)
        simulated_data.loc[approved_mask, 'balance'] = np.minimum(
            simulated_data.loc[approved_mask, 'balance'] * multiplier,
            simulated_data.loc[approved_mask, 'credit_limit']
        )
        
        # Recalculate utilization
        simulated_data.loc[approved_mask, 'utilization_rate'] = (
            simulated_data.loc[approved_mask, 'balance'] / 
            simulated_data.loc[approved_mask, 'credit_limit'] * 100
        )
        
        # Adjust risk metrics (higher limits = higher EAD)
        if 'ead' in simulated_data.columns:
            simulated_data.loc[approved_mask, 'ead'] = simulated_data.loc[approved_mask, 'ead'] * multiplier
            simulated_data.loc[approved_mask, 'expected_loss'] = (
                simulated_data.loc[approved_mask, 'pd_12m'] * 
                simulated_data.loc[approved_mask, 'lgd'] * 
                simulated_data.loc[approved_mask, 'ead']
            )
        
        approved = simulated_data[simulated_data['acceptance_decision'] == 'Approved']
        
        return {
            'strategy': f'Credit Limit {multiplier:.1f}x',
            'customers': len(approved),
            'approval_rate': len(approved) / len(self.data) * 100,
            'portfolio_balance': approved['balance'].sum(),
            'credit_limits': approved['credit_limit'].sum(),
            'delinquency_rate': (approved['delinquency_status'] > 0).mean() * 100,
            'expected_loss': approved['expected_loss'].sum() if 'expected_loss' in approved.columns else 0,
            'loss_rate': approved['expected_loss'].sum() / approved['balance'].sum() * 100 if 'expected_loss' in approved.columns and approved['balance'].sum() > 0 else 0,
            'avg_score': approved['application_score'].mean(),
            'revenue_estimate': approved['balance'].sum() * 0.18 / 12,
            'net_income': approved['balance'].sum() * 0.18 / 12 - (approved['expected_loss'].sum() if 'expected_loss' in approved.columns else 0),
            'roa': ((approved['balance'].sum() * 0.18 / 12 - (approved['expected_loss'].sum() if 'expected_loss' in approved.columns else 0)) / approved['balance'].sum() * 100) if approved['balance'].sum() > 0 else 0
        }
    
    def simulate_income_targeting(self, target_income_bands):
        """
        Simulate targeting specific income bands only
        """
        simulated_data = self.data.copy()
        
        # Apply income band filter
        simulated_data['new_acceptance'] = 'Declined'
        eligible_mask = (
            simulated_data['income_band'].isin(target_income_bands) & 
            (simulated_data['acceptance_decision'] == 'Approved')
        )
        simulated_data.loc[eligible_mask, 'new_acceptance'] = 'Approved'
        
        approved = simulated_data[simulated_data['new_acceptance'] == 'Approved']
        
        if len(approved) == 0:
            return self._empty_result()
        
        return {
            'strategy': f'Target Income: {", ".join(target_income_bands)}',
            'customers': len(approved),
            'approval_rate': len(approved) / len(self.data) * 100,
            'portfolio_balance': approved['balance'].sum(),
            'credit_limits': approved['credit_limit'].sum(),
            'delinquency_rate': (approved['delinquency_status'] > 0).mean() * 100,
            'expected_loss': approved['expected_loss'].sum(),
            'loss_rate': approved['expected_loss'].sum() / approved['balance'].sum() * 100 if approved['balance'].sum() > 0 else 0,
            'avg_score': approved['application_score'].mean(),
            'revenue_estimate': approved['balance'].sum() * 0.18 / 12,
            'net_income': approved['balance'].sum() * 0.18 / 12 - approved['expected_loss'].sum(),
            'roa': (approved['balance'].sum() * 0.18 / 12 - approved['expected_loss'].sum()) / approved['balance'].sum() * 100 if approved['balance'].sum() > 0 else 0
        }
    
    def _empty_result(self):
        """Return empty result for invalid strategies"""
        return {
            'strategy': 'Invalid Strategy',
            'customers': 0,
            'approval_rate': 0,
            'portfolio_balance': 0,
            'credit_limits': 0,
            'delinquency_rate': 0,
            'expected_loss': 0,
            'loss_rate': 0,
            'avg_score': 0,
            'revenue_estimate': 0,
            'net_income': 0,
            'roa': 0
        }

# Initialize simulator
simulator = CreditStrategySimulator(strategy_df)
print("Strategy simulator initialized")
print(f"Baseline portfolio: {simulator.baseline['customers']:,} customers")
print(f"Baseline ROA: {simulator.baseline['roa']:.2f}%")

## 2. Score Threshold Strategy Analysis

Analyze impact of tightening or loosening acceptance score criteria.

In [ ]:
# Test different score thresholds
score_thresholds = [400, 450, 500, 550, 600, 650, 700, 750]
score_results = []

for threshold in score_thresholds:
    result = simulator.simulate_score_threshold_strategy(threshold)
    result['threshold'] = threshold
    score_results.append(result)

score_df = pd.DataFrame(score_results)

print("=== SCORE THRESHOLD STRATEGY RESULTS ===")
display_cols = ['threshold', 'customers', 'approval_rate', 'delinquency_rate', 'loss_rate', 'roa']
print(score_df[display_cols].round(2))

In [ ]:
# Visualize score threshold trade-offs
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Credit Score Threshold Strategy Analysis', fontsize=16, fontweight='bold')

# Volume vs Quality Trade-off
ax1 = axes[0, 0]
ax1_twin = ax1.twinx()

line1 = ax1.plot(score_df['threshold'], score_df['approval_rate'], 'b-o', label='Approval Rate')
line2 = ax1_twin.plot(score_df['threshold'], score_df['delinquency_rate'], 'r-s', label='Delinquency Rate')

ax1.set_xlabel('Score Threshold')
ax1.set_ylabel('Approval Rate (%)', color='b')
ax1_twin.set_ylabel('Delinquency Rate (%)', color='r')
ax1.set_title('Volume vs Risk Trade-off')
ax1.grid(True, alpha=0.3)

# Add current baseline marker
current_approval = simulator.baseline['approval_rate']
current_delinq = simulator.baseline['delinquency_rate']
ax1.axhline(current_approval, color='blue', linestyle='--', alpha=0.5, label='Current Approval Rate')
ax1_twin.axhline(current_delinq, color='red', linestyle='--', alpha=0.5, label='Current Delinquency Rate')

# Portfolio Balance Impact
axes[0, 1].plot(score_df['threshold'], score_df['portfolio_balance'] / 1e6, 'g-o')
axes[0, 1].axhline(simulator.baseline['portfolio_balance'] / 1e6, color='green', linestyle='--', alpha=0.5, label='Current Balance')
axes[0, 1].set_xlabel('Score Threshold')
axes[0, 1].set_ylabel('Portfolio Balance (£M)')
axes[0, 1].set_title('Portfolio Size Impact')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# ROA Impact
axes[1, 0].plot(score_df['threshold'], score_df['roa'], 'purple', marker='o', linewidth=2)
axes[1, 0].axhline(simulator.baseline['roa'], color='purple', linestyle='--', alpha=0.5, label='Current ROA')
axes[1, 0].set_xlabel('Score Threshold')
axes[1, 0].set_ylabel('ROA (%)')
axes[1, 0].set_title('Return on Assets')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

# Expected Loss Rate
axes[1, 1].plot(score_df['threshold'], score_df['loss_rate'], 'orange', marker='s', linewidth=2)
axes[1, 1].axhline(simulator.baseline['loss_rate'], color='orange', linestyle='--', alpha=0.5, label='Current Loss Rate')
axes[1, 1].set_xlabel('Score Threshold')
axes[1, 1].set_ylabel('Loss Rate (%)')
axes[1, 1].set_title('Portfolio Loss Rate')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Find optimal score threshold (maximize ROA)
optimal_score_idx = score_df['roa'].idxmax()
optimal_threshold = score_df.loc[optimal_score_idx, 'threshold']
optimal_roa = score_df.loc[optimal_score_idx, 'roa']
roa_improvement = optimal_roa - simulator.baseline['roa']

print(f"\n=== OPTIMAL SCORE THRESHOLD STRATEGY ===")
print(f"Optimal Score Threshold: {optimal_threshold}")
print(f"ROA Improvement: {roa_improvement:.2f}% ({optimal_roa:.2f}% vs {simulator.baseline['roa']:.2f}%)")
print(f"Approval Rate Impact: {score_df.loc[optimal_score_idx, 'approval_rate']:.1f}% vs {simulator.baseline['approval_rate']:.1f}%")
print(f"Delinquency Improvement: {score_df.loc[optimal_score_idx, 'delinquency_rate']:.1f}% vs {simulator.baseline['delinquency_rate']:.1f}%")

## 3. Credit Limit Strategy Analysis

Analyze impact of credit limit policy changes.

In [ ]:
# Test different credit limit multipliers
limit_multipliers = [0.5, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.5]
limit_results = []

for multiplier in limit_multipliers:
    result = simulator.simulate_limit_multiplier_strategy(multiplier)
    result['multiplier'] = multiplier
    limit_results.append(result)

limit_df = pd.DataFrame(limit_results)

print("=== CREDIT LIMIT STRATEGY RESULTS ===")
display_cols = ['multiplier', 'portfolio_balance', 'credit_limits', 'loss_rate', 'roa']
print(limit_df[display_cols].round(2))

In [ ]:
# Visualize credit limit strategy impact
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Credit Limit Strategy Analysis', fontsize=16, fontweight='bold')

# Portfolio Growth vs Risk
ax1 = axes[0, 0]
ax1_twin = ax1.twinx()

line1 = ax1.plot(limit_df['multiplier'], limit_df['portfolio_balance'] / 1e6, 'b-o', label='Portfolio Balance (£M)')
line2 = ax1_twin.plot(limit_df['multiplier'], limit_df['loss_rate'], 'r-s', label='Loss Rate (%)')

ax1.set_xlabel('Credit Limit Multiplier')
ax1.set_ylabel('Portfolio Balance (£M)', color='b')
ax1_twin.set_ylabel('Loss Rate (%)', color='r')
ax1.set_title('Portfolio Growth vs Risk')
ax1.grid(True, alpha=0.3)
ax1.axvline(1.0, color='black', linestyle='--', alpha=0.5, label='Current Policy')

# Credit Utilization Impact
axes[0, 1].plot(limit_df['multiplier'], limit_df['credit_limits'] / 1e6, 'g-o', label='Total Credit Limits')
axes[0, 1].plot(limit_df['multiplier'], limit_df['portfolio_balance'] / 1e6, 'b-s', label='Portfolio Balance')
axes[0, 1].axvline(1.0, color='black', linestyle='--', alpha=0.5, label='Current Policy')
axes[0, 1].set_xlabel('Credit Limit Multiplier')
axes[0, 1].set_ylabel('Amount (£M)')
axes[0, 1].set_title('Credit Capacity vs Usage')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# ROA vs Revenue Trade-off
axes[1, 0].plot(limit_df['multiplier'], limit_df['roa'], 'purple', marker='o', linewidth=2, label='ROA')
axes[1, 0].axvline(1.0, color='black', linestyle='--', alpha=0.5)
axes[1, 0].axhline(simulator.baseline['roa'], color='purple', linestyle='--', alpha=0.5, label='Current ROA')
axes[1, 0].set_xlabel('Credit Limit Multiplier')
axes[1, 0].set_ylabel('ROA (%)')
axes[1, 0].set_title('Return on Assets')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Revenue Impact
axes[1, 1].plot(limit_df['multiplier'], limit_df['revenue_estimate'] / 1e6, 'teal', marker='s', linewidth=2)
axes[1, 1].axvline(1.0, color='black', linestyle='--', alpha=0.5, label='Current Policy')
axes[1, 1].axhline(simulator.baseline['revenue_estimate'] / 1e6, color='teal', linestyle='--', alpha=0.5, label='Current Revenue')
axes[1, 1].set_xlabel('Credit Limit Multiplier')
axes[1, 1].set_ylabel('Monthly Revenue (£M)')
axes[1, 1].set_title('Revenue Impact')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find optimal limit multiplier
optimal_limit_idx = limit_df['roa'].idxmax()
optimal_multiplier = limit_df.loc[optimal_limit_idx, 'multiplier']
optimal_limit_roa = limit_df.loc[optimal_limit_idx, 'roa']
limit_roa_improvement = optimal_limit_roa - simulator.baseline['roa']

print(f"\n=== OPTIMAL CREDIT LIMIT STRATEGY ===")
print(f"Optimal Limit Multiplier: {optimal_multiplier:.1f}x")
print(f"ROA Impact: {limit_roa_improvement:+.2f}% ({optimal_limit_roa:.2f}% vs {simulator.baseline['roa']:.2f}%)")
portfolio_change = (limit_df.loc[optimal_limit_idx, 'portfolio_balance'] - simulator.baseline['portfolio_balance']) / simulator.baseline['portfolio_balance'] * 100
print(f"Portfolio Growth: {portfolio_change:+.1f}%")

## 4. Income Band Targeting Strategy

Analyze focus on specific customer segments.

In [ ]:
# Test different income targeting strategies
income_strategies = [
    ['High'],
    ['Medium', 'High'], 
    ['Low', 'Medium', 'High'],  # Current baseline
    ['Medium'],
    ['Low']
]

income_results = []
for strategy in income_strategies:
    result = simulator.simulate_income_targeting(strategy)
    result['income_bands'] = ', '.join(strategy)
    income_results.append(result)

income_df = pd.DataFrame(income_results)

print("=== INCOME TARGETING STRATEGY RESULTS ===")
display_cols = ['income_bands', 'customers', 'approval_rate', 'delinquency_rate', 'loss_rate', 'roa']
print(income_df[display_cols].round(2))

# Visualize income targeting results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Income Band Targeting Strategy Analysis', fontsize=16, fontweight='bold')

strategies = income_df['income_bands'].tolist()
x_pos = range(len(strategies))

# Customer Volume
axes[0, 0].bar(x_pos, income_df['customers'] / 1000, alpha=0.7, color='skyblue')
axes[0, 0].set_xlabel('Strategy')
axes[0, 0].set_ylabel('Customers (000s)')
axes[0, 0].set_title('Customer Volume by Strategy')
axes[0, 0].set_xticks(x_pos)
axes[0, 0].set_xticklabels(strategies, rotation=45, ha='right')
axes[0, 0].grid(True, alpha=0.3)

# Risk Quality
axes[0, 1].bar(x_pos, income_df['delinquency_rate'], alpha=0.7, color='red')
axes[0, 1].set_xlabel('Strategy')
axes[0, 1].set_ylabel('Delinquency Rate (%)')
axes[0, 1].set_title('Risk Quality by Strategy')
axes[0, 1].set_xticks(x_pos)
axes[0, 1].set_xticklabels(strategies, rotation=45, ha='right')
axes[0, 1].grid(True, alpha=0.3)

# ROA Comparison
colors = ['green' if roa > simulator.baseline['roa'] else 'red' for roa in income_df['roa']]
axes[1, 0].bar(x_pos, income_df['roa'], alpha=0.7, color=colors)
axes[1, 0].axhline(simulator.baseline['roa'], color='black', linestyle='--', alpha=0.7, label='Baseline ROA')
axes[1, 0].set_xlabel('Strategy')
axes[1, 0].set_ylabel('ROA (%)')
axes[1, 0].set_title('Return on Assets by Strategy')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels(strategies, rotation=45, ha='right')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Portfolio Balance
axes[1, 1].bar(x_pos, income_df['portfolio_balance'] / 1e6, alpha=0.7, color='purple')
axes[1, 1].axhline(simulator.baseline['portfolio_balance'] / 1e6, color='black', linestyle='--', alpha=0.7, label='Baseline Balance')
axes[1, 1].set_xlabel('Strategy')
axes[1, 1].set_ylabel('Portfolio Balance (£M)')
axes[1, 1].set_title('Portfolio Size by Strategy')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(strategies, rotation=45, ha='right')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Best income targeting strategy
best_income_idx = income_df['roa'].idxmax()
best_income_strategy = income_df.loc[best_income_idx, 'income_bands']
best_income_roa = income_df.loc[best_income_idx, 'roa']

print(f"\n=== OPTIMAL INCOME TARGETING STRATEGY ===")
print(f"Best Strategy: {best_income_strategy}")
print(f"ROA: {best_income_roa:.2f}% vs Baseline {simulator.baseline['roa']:.2f}%")
print(f"Customer Impact: {income_df.loc[best_income_idx, 'customers']:,} vs {simulator.baseline['customers']:,}")

## 5. Combined Strategy Optimization

Test combinations of strategies for maximum impact.

In [ ]:
# Test combined strategies
def simulate_combined_strategy(data, min_score, limit_multiplier, target_income_bands):
    """
    Simulate combined strategy with multiple levers
    """
    simulated_data = data.copy()
    
    # Apply score threshold
    score_eligible = simulated_data['application_score'] >= min_score
    
    # Apply income targeting
    income_eligible = simulated_data['income_band'].isin(target_income_bands)
    
    # Combined acceptance criteria
    simulated_data['new_acceptance'] = 'Declined'
    eligible_mask = score_eligible & income_eligible
    simulated_data.loc[eligible_mask, 'new_acceptance'] = 'Approved'
    
    # Apply credit limit multiplier to approved customers
    approved_mask = simulated_data['new_acceptance'] == 'Approved'
    if approved_mask.any():
        simulated_data.loc[approved_mask, 'credit_limit'] = simulated_data.loc[approved_mask, 'credit_limit'] * limit_multiplier
        
        # Adjust balance (assume similar utilization behavior)
        simulated_data.loc[approved_mask, 'balance'] = np.minimum(
            simulated_data.loc[approved_mask, 'balance'] * limit_multiplier,
            simulated_data.loc[approved_mask, 'credit_limit']
        )
        
        # Recalculate utilization
        simulated_data.loc[approved_mask, 'utilization_rate'] = (
            simulated_data.loc[approved_mask, 'balance'] / 
            simulated_data.loc[approved_mask, 'credit_limit'] * 100
        )
        
        # Adjust risk metrics
        if 'ead' in simulated_data.columns:
            simulated_data.loc[approved_mask, 'ead'] = simulated_data.loc[approved_mask, 'ead'] * limit_multiplier
            simulated_data.loc[approved_mask, 'expected_loss'] = (
                simulated_data.loc[approved_mask, 'pd_12m'] * 
                simulated_data.loc[approved_mask, 'lgd'] * 
                simulated_data.loc[approved_mask, 'ead']
            )
    
    approved = simulated_data[simulated_data['new_acceptance'] == 'Approved']
    
    if len(approved) == 0:
        return {
            'customers': 0, 'portfolio_balance': 0, 'expected_loss': 0,
            'delinquency_rate': 0, 'roa': 0, 'approval_rate': 0
        }
    
    total_customers = len(simulated_data)
    portfolio_balance = approved['balance'].sum()
    expected_loss = approved['expected_loss'].sum() if 'expected_loss' in approved.columns else 0
    revenue = portfolio_balance * 0.18 / 12
    
    return {
        'customers': len(approved),
        'approval_rate': len(approved) / total_customers * 100,
        'portfolio_balance': portfolio_balance,
        'delinquency_rate': (approved['delinquency_status'] > 0).mean() * 100,
        'expected_loss': expected_loss,
        'loss_rate': expected_loss / portfolio_balance * 100 if portfolio_balance > 0 else 0,
        'revenue': revenue,
        'net_income': revenue - expected_loss,
        'roa': (revenue - expected_loss) / portfolio_balance * 100 if portfolio_balance > 0 else 0,
        'avg_score': approved['application_score'].mean()
    }

# Test key combined strategies
combined_strategies = [
    {'name': 'Conservative Quality', 'score': 650, 'multiplier': 0.9, 'income': ['Medium', 'High']},
    {'name': 'Aggressive Growth', 'score': 550, 'multiplier': 1.3, 'income': ['Low', 'Medium', 'High']},
    {'name': 'Premium Focus', 'score': 700, 'multiplier': 1.2, 'income': ['High']},
    {'name': 'Balanced Optimization', 'score': 600, 'multiplier': 1.1, 'income': ['Medium', 'High']},
    {'name': 'Current Baseline', 'score': 500, 'multiplier': 1.0, 'income': ['Low', 'Medium', 'High']}
]

combined_results = []
for strategy in combined_strategies:
    result = simulate_combined_strategy(
        strategy_df, 
        strategy['score'], 
        strategy['multiplier'], 
        strategy['income']
    )
    result['strategy_name'] = strategy['name']
    result['score_threshold'] = strategy['score']
    result['limit_multiplier'] = strategy['multiplier']
    result['target_income'] = ', '.join(strategy['income'])
    combined_results.append(result)

combined_df = pd.DataFrame(combined_results)

print("=== COMBINED STRATEGY RESULTS ===")
display_cols = ['strategy_name', 'customers', 'approval_rate', 'delinquency_rate', 'roa', 'portfolio_balance']
display_df = combined_df[display_cols].copy()
display_df['portfolio_balance'] = display_df['portfolio_balance'] / 1e6  # Convert to millions
print(display_df.round(2))

In [ ]:
# Visualize combined strategy results
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Combined Strategy Analysis - ROA vs Risk vs Growth', fontsize=16, fontweight='bold')

strategies = combined_df['strategy_name'].tolist()
colors = ['red', 'green', 'blue', 'orange', 'purple']
x_pos = range(len(strategies))

# ROA Comparison
bars1 = axes[0, 0].bar(x_pos, combined_df['roa'], color=colors, alpha=0.7)
axes[0, 0].axhline(simulator.baseline['roa'], color='black', linestyle='--', linewidth=2, label='Baseline ROA')
axes[0, 0].set_xlabel('Strategy')
axes[0, 0].set_ylabel('ROA (%)')
axes[0, 0].set_title('Return on Assets Comparison')
axes[0, 0].set_xticks(x_pos)
axes[0, 0].set_xticklabels(strategies, rotation=45, ha='right')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Add values on bars
for i, bar in enumerate(bars1):
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + 0.1, f'{height:.1f}%', 
                    ha='center', va='bottom')

# Risk vs Volume Trade-off (Scatter)
axes[0, 1].scatter(combined_df['delinquency_rate'], combined_df['customers'] / 1000, 
                   c=colors, s=100, alpha=0.7)
for i, strategy in enumerate(strategies):
    axes[0, 1].annotate(strategy, 
                        (combined_df.loc[i, 'delinquency_rate'], combined_df.loc[i, 'customers'] / 1000),
                        xytext=(5, 5), textcoords='offset points', fontsize=8)
axes[0, 1].set_xlabel('Delinquency Rate (%)')
axes[0, 1].set_ylabel('Customer Volume (000s)')
axes[0, 1].set_title('Risk vs Volume Trade-off')
axes[0, 1].grid(True, alpha=0.3)

# Portfolio Growth Impact
baseline_balance = simulator.baseline['portfolio_balance'] / 1e6
balance_changes = (combined_df['portfolio_balance'] / 1e6 - baseline_balance) / baseline_balance * 100
bar_colors = ['green' if x > 0 else 'red' if x < 0 else 'gray' for x in balance_changes]

bars3 = axes[1, 0].bar(x_pos, balance_changes, color=bar_colors, alpha=0.7)
axes[1, 0].axhline(0, color='black', linestyle='-', alpha=0.5)
axes[1, 0].set_xlabel('Strategy')
axes[1, 0].set_ylabel('Portfolio Growth (%)')
axes[1, 0].set_title('Portfolio Growth vs Baseline')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels(strategies, rotation=45, ha='right')
axes[1, 0].grid(True, alpha=0.3)

# Add values on bars
for i, bar in enumerate(bars3):
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + (1 if height > 0 else -3), 
                    f'{height:+.1f}%', ha='center', va='bottom' if height > 0 else 'top')

# Efficiency Frontier: ROA vs Risk
axes[1, 1].scatter(combined_df['loss_rate'], combined_df['roa'], c=colors, s=150, alpha=0.7)
for i, strategy in enumerate(strategies):
    axes[1, 1].annotate(strategy, 
                        (combined_df.loc[i, 'loss_rate'], combined_df.loc[i, 'roa']),
                        xytext=(5, 5), textcoords='offset points', fontsize=8)
axes[1, 1].set_xlabel('Loss Rate (%)')
axes[1, 1].set_ylabel('ROA (%)')
axes[1, 1].set_title('Efficiency Frontier: ROA vs Loss Rate')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Strategy ranking
strategy_ranking = combined_df.sort_values('roa', ascending=False)[['strategy_name', 'roa', 'customers', 'portfolio_balance']].copy()
strategy_ranking['portfolio_balance'] = strategy_ranking['portfolio_balance'] / 1e6
strategy_ranking['rank'] = range(1, len(strategy_ranking) + 1)

print(f"\n=== STRATEGY RANKING (BY ROA) ===")
print(strategy_ranking[['rank', 'strategy_name', 'roa', 'customers', 'portfolio_balance']].round(2))

## 6. Strategy Recommendations & Business Impact

Synthesize findings into actionable recommendations.

In [ ]:
# Calculate comprehensive impact analysis for top strategies
def calculate_comprehensive_impact(strategy_result, baseline):
    """
    Calculate comprehensive business impact metrics
    """
    impact = {}
    
    # Volume metrics
    impact['customer_change'] = strategy_result['customers'] - baseline['customers']
    impact['customer_change_pct'] = (strategy_result['customers'] - baseline['customers']) / baseline['customers'] * 100
    
    # Balance metrics
    impact['balance_change'] = strategy_result['portfolio_balance'] - baseline['portfolio_balance']
    impact['balance_change_pct'] = (strategy_result['portfolio_balance'] - baseline['portfolio_balance']) / baseline['portfolio_balance'] * 100
    
    # Profitability metrics
    impact['roa_change'] = strategy_result['roa'] - baseline['roa']
    impact['net_income_change'] = strategy_result['net_income'] - baseline['net_income']
    impact['annual_net_income_impact'] = impact['net_income_change'] * 12  # Annualized
    
    # Risk metrics
    impact['delinq_change'] = strategy_result['delinquency_rate'] - baseline['delinquency_rate']
    impact['loss_rate_change'] = strategy_result['loss_rate'] - baseline['loss_rate']
    impact['expected_loss_change'] = strategy_result['expected_loss'] - baseline['expected_loss']
    impact['annual_loss_impact'] = impact['expected_loss_change'] * 12  # Annualized
    
    return impact

# Analyze top 3 strategies
top_strategies = combined_df.nlargest(3, 'roa')

print("=== COMPREHENSIVE STRATEGY IMPACT ANALYSIS ===")
print("\n" + "="*80)

for idx, row in top_strategies.iterrows():
    impact = calculate_comprehensive_impact(row, simulator.baseline)
    
    print(f"\n{row['strategy_name'].upper()} STRATEGY")
    print(f"Score Threshold: {row['score_threshold']} | Limit Multiplier: {row['limit_multiplier']:.1f}x | Target: {row['target_income']}")
    print(f"-" * 60)
    
    print(f"VOLUME IMPACT:")
    print(f"  • Customers: {impact['customer_change']:+,} ({impact['customer_change_pct']:+.1f}%)")
    print(f"  • Portfolio Balance: £{impact['balance_change']/1e6:+.1f}M ({impact['balance_change_pct']:+.1f}%)")
    
    print(f"\nPROFITABILITY IMPACT:")
    print(f"  • ROA Change: {impact['roa_change']:+.2f}% ({row['roa']:.2f}% vs {simulator.baseline['roa']:.2f}%)")
    print(f"  • Annual Net Income Impact: £{impact['annual_net_income_impact']/1e6:+.2f}M")
    
    print(f"\nRISK IMPACT:")
    print(f"  • Delinquency Rate: {impact['delinq_change']:+.1f}% ({row['delinquency_rate']:.1f}% vs {simulator.baseline['delinquency_rate']:.1f}%)")
    print(f"  • Loss Rate: {impact['loss_rate_change']:+.2f}% ({row['loss_rate']:.2f}% vs {simulator.baseline['loss_rate']:.2f}%)")
    print(f"  • Annual Loss Impact: £{impact['annual_loss_impact']/1e6:+.2f}M")
    
    # ROI calculation
    roi = impact['annual_net_income_impact'] / abs(impact['balance_change']) * 100 if impact['balance_change'] != 0 else 0
    print(f"\nSTRATEGY ROI: {roi:.1f}% (Annual Net Income / Balance Investment)")
    
    print(f"\n" + "="*60)

In [ ]:
# Create final strategy recommendation summary
best_strategy = combined_df.loc[combined_df['roa'].idxmax()]
best_impact = calculate_comprehensive_impact(best_strategy, simulator.baseline)

recommendation_summary = {
    'strategy_name': best_strategy['strategy_name'],
    'score_threshold': best_strategy['score_threshold'],
    'limit_multiplier': best_strategy['limit_multiplier'],
    'target_segments': best_strategy['target_income'],
    'roa_improvement': best_impact['roa_change'],
    'annual_profit_impact': best_impact['annual_net_income_impact'],
    'risk_improvement': -best_impact['delinq_change'],  # Negative because we want improvement
    'portfolio_growth': best_impact['balance_change_pct'],
    'customer_impact': best_impact['customer_change_pct']
}

print("\n" + "="*80)
print("EXECUTIVE SUMMARY - OPTIMAL CREDIT STRATEGY")
print("="*80)

print(f"\nRECOMMENDED STRATEGY: {recommendation_summary['strategy_name']}")
print(f"\nKEY PARAMETERS:")
print(f"  • Minimum Credit Score: {recommendation_summary['score_threshold']}")
print(f"  • Credit Limit Policy: {recommendation_summary['limit_multiplier']:.1f}x current limits")
print(f"  • Target Customer Segments: {recommendation_summary['target_segments']}")

print(f"\nBUSINESS IMPACT:")
print(f"  • ROA Improvement: +{recommendation_summary['roa_improvement']:.2f}% ({best_strategy['roa']:.2f}% vs {simulator.baseline['roa']:.2f}%)")
print(f"  • Annual Profit Impact: £{recommendation_summary['annual_profit_impact']/1e6:.2f}M additional net income")
print(f"  • Risk Reduction: {recommendation_summary['risk_improvement']:.1f}% lower delinquency rate")
print(f"  • Portfolio Growth: {recommendation_summary['portfolio_growth']:+.1f}%")
print(f"  • Customer Base: {recommendation_summary['customer_impact']:+.1f}% change")

print(f"\nIMPLEMENTATION BENEFITS:")
if recommendation_summary['roa_improvement'] > 0:
    print(f"  ✓ Improved portfolio profitability")
if recommendation_summary['risk_improvement'] > 0:
    print(f"  ✓ Enhanced risk quality")
if abs(recommendation_summary['customer_impact']) < 50:
    print(f"  ✓ Sustainable customer base impact")

print(f"\nNEXT STEPS:")
print(f"  1. Implement {recommendation_summary['strategy_name']} strategy")
print(f"  2. Monitor portfolio performance over 3-6 months")
print(f"  3. Fine-tune parameters based on actual results")
print(f"  4. Consider A/B testing for validation")

print("\n" + "="*80)

In [ ]:
# Save strategy analysis results
# Score threshold analysis
score_df.to_csv('../data/score_threshold_analysis.csv', index=False)

# Credit limit analysis
limit_df.to_csv('../data/credit_limit_analysis.csv', index=False)

# Income targeting analysis
income_df.to_csv('../data/income_targeting_analysis.csv', index=False)

# Combined strategy results
combined_df.to_csv('../data/combined_strategy_analysis.csv', index=False)

# Strategy recommendation summary
recommendation_df = pd.DataFrame([recommendation_summary])
recommendation_df.to_csv('../data/strategy_recommendation.csv', index=False)

print("=== STRATEGY ANALYSIS COMPLETE ===")
print(f"✓ Analyzed {len(score_thresholds)} score threshold scenarios")
print(f"✓ Tested {len(limit_multipliers)} credit limit policies")
print(f"✓ Evaluated {len(income_strategies)} income targeting strategies")
print(f"✓ Compared {len(combined_strategies)} combined strategies")
print(f"✓ Identified optimal strategy: {best_strategy['strategy_name']}")
print(f"✓ Projected annual profit impact: £{best_impact['annual_net_income_impact']/1e6:.2f}M")
print(f"✓ Strategy analysis results saved to ../data/")

print(f"\n=== FINAL RECOMMENDATION ===")
print(f"Implement '{best_strategy['strategy_name']}' for +{best_impact['roa_change']:.2f}% ROA improvement")
print(f"Expected annual benefit: £{best_impact['annual_net_income_impact']/1e6:.2f}M")